<a href="https://colab.research.google.com/github/tozanni/Deep_Learning_Notebooks/blob/main/DL_LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generación de texto sintético con una red LSTM

Referencia: https://gist.github.com/maxim5/c35ef2238ae708ccb0e55624e9e0252b


In [30]:
from __future__ import print_function
import numpy as np
import gensim
import string

from keras.callbacks import LambdaCallback
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import get_file


In [31]:
## Descargar archivo de abstracts de Stanford

# Original
# https://raw.githubusercontent.com/maxim5/stanford-tensorflow-tutorials/master/data/arxiv_abstracts.txt

# Local
url = 'https://raw.githubusercontent.com/tozanni/Data_Science_Notebooks/main/arxiv_abstracts.txt'
path = get_file('arxiv_abstracts.txt', origin=url)


In [32]:
## Generar sentencias de longitud 40
max_sentence_len = 40
with open(path) as file_:
  docs = file_.readlines()
  sentences = [[word for word in doc.lower().split()[:max_sentence_len]] for doc in docs]

print('Num sentences:', len(sentences))

Num sentences: 7200


### Vectorización con Word2Vec

A continuación se entrenará el modelo de embeddings Word2Vec, dicho modelo nos permitirá representar nuestras palabras en vectores que mantienen ciertas propiedades de similaridad semántica en sus dimensiones.

In [33]:
def word2idx(word):
  return word_model.wv.key_to_index[word]

def idx2word(idx):
  return word_model.wv.index_to_key[idx]

In [34]:

print('Entrenando modelo word2vec con 100 dimensiones...')
word_model = gensim.models.Word2Vec(sentences, vector_size=100, min_count=1, window=5, epochs=100)
pretrained_weights = word_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

print('Obtener palabras similares a algunos ejemplos:')
for word in ['model', 'network', 'train', 'learn']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))


Entrenando modelo word2vec con 100 dimensiones...
Result embedding shape: (1350, 100)
Obtener palabras similares a algunos ejemplos:
  model -> $l_p$ (0.40), technique (0.37), architecture. (0.32), trains (0.31), framework (0.31), continuous (0.30), possible (0.30), 2012) (0.29)
  network -> networks (0.41), constrained (0.27), trained (0.26), guide (0.25), networks. (0.24), function (0.24), broad (0.24), help (0.24)
  train -> based (0.37), directly (0.33), eigendecompositions (0.32), classical (0.32), sequentially (0.32), represent (0.31), tend (0.31), map (0.29)
  learn -> remain (0.38), tend (0.36), adapt (0.36), adopted (0.36), automatically (0.35), effectively (0.35), lower (0.33), enormous (0.33)


### Creación de training y test set para LSTM

In [35]:
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)

for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

# Ejemplo de datos de training y test
# Nuestras secuencias de training y test son los índices
# de las palabras del diccionario

train_x[0], train_y[0]


train_x shape: (7200, 40)
train_y shape: (7200,)


(array([  4, 555,   5, 556, 557, 136,   1, 117, 151,  26,  19, 239, 558,
         50, 415,  11, 120, 416,  18,   2, 561, 417,   1, 245, 136, 418,
        568, 569, 554, 205, 552, 551,  58, 538, 539, 540, 541,   2, 421,
          0], dtype=int32),
 3)

## Definición y entrenamiento de la red LSTM

1. Notar que la primera capa corresponde a la capa de embeddings, de tal forma que el primer proceso de la red será mapear el input al espacio de los embeddings.

2. Posteriormente se aplica la capa LSTM con una cantidad de unidades arbitraria definida por nosotros.

3. Finalmente, pasamos a una capa densa con tantas neuronas como palabras e nuestro vocabulario con activación Softmax para generar como output una distribución de probabilidades de la siguiente palabra.

In [36]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))

model.add(LSTM(units=16))

#Modelo con dos capas anidadas recurrentes
#model.add(LSTM(units=32, return_sequences=True))
#model.add(LSTM(units=16))

#Probar otras definiciones de units
#model.add(LSTM(units=emdedding_size))

model.add(Dense(units=vocab_size))
model.add(Activation('softmax')) #El resultado es un vector de probabilidades
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.fit(train_x, train_y,
          batch_size=128,
          epochs=20)

Epoch 1/20
57/57 [==============================] - 5s 41ms/step - loss: 6.6294
Epoch 2/20
57/57 [==============================] - 3s 52ms/step - loss: 5.1168
Epoch 3/20
57/57 [==============================] - 2s 36ms/step - loss: 4.2837
Epoch 4/20
57/57 [==============================] - 2s 36ms/step - loss: 3.8921
Epoch 5/20
57/57 [==============================] - 2s 36ms/step - loss: 3.6549
Epoch 6/20
57/57 [==============================] - 2s 35ms/step - loss: 3.4749
Epoch 7/20
57/57 [==============================] - 2s 43ms/step - loss: 3.3135
Epoch 8/20
57/57 [==============================] - 3s 51ms/step - loss: 3.1696
Epoch 9/20
57/57 [==============================] - 2s 42ms/step - loss: 3.0363
Epoch 10/20
57/57 [==============================] - 3s 52ms/step - loss: 2.9102
Epoch 11/20
57/57 [==============================] - 2s 36ms/step - loss: 2.7841
Epoch 12/20
57/57 [==============================] - 2s 39ms/step - loss: 2.6596
Epoch 13/20
57/57 [==================

### Generación de texto con el modelo LSTM

Ahora hay que generar texto sintético, en esta etapa utilizaremos el modelo de forma iterativa comenzando por una semilla, posteriormente concatenando el output de cada etapa y pasandolo a la siguiente iteración.

Notar que el modelo entrega una distribución de probabilidad de las siguientes palabras más probables y no es idóneo elegir siempre la mayor (ej. usando argmax) por lo cual se recomienda aplicar un método de sampling sobre dicha distribución.


Referencia de sampling:

https://medium.com/machine-learning-at-petiteprogrammer/sampling-strategies-for-recurrent-neural-networks-9aea02a6616f


In [37]:
def sample(preds, temperature=1.0):
  """
  Metodo de muestreo aleatorio de siguiente palabra.
  Toma como input la distribucion de probabilidad entregada por la red.
  Con cierta proabilidad dependiendo de la temperatura produce la
  siguiente palabra.
  """

  if temperature <= 0:
    return np.argmax(preds)

  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):

    #El input se incrementa en cada iteracion a la RNN
    print("Input --> ",word_idxs)
    x=np.array(word_idxs)

    #Tenemos que convertir el input a 3D agregando una dimension dummy
    x=np.expand_dims(x,1)
    prediction = model.predict(x)
    print("Prediction -->", prediction)

    #No realizar sampling, tomar la palabra con mayor probabilidad
    #idx = np.argmax(prediction[-1])

    #Realizar un muestreo aleatorio
    idx = sample(prediction[-1], temperature=0.7)

    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)


In [38]:
next_words = 20 #Cuantas palabras se generaran?

generated_text = generate_next('deep convolutional', next_words)


Input -->  [6, 37]
1/1 [==============================] - 0s 479ms/step
Prediction --> [[0.01500825 0.00495367 0.00821734 ... 0.00049222 0.00044217 0.00047735]
 [0.00491441 0.00179882 0.00475018 ... 0.00060894 0.00064152 0.00063278]]
Input -->  [6, 37, 347]
1/1 [==============================] - 0s 27ms/step
Prediction --> [[0.01500825 0.00495367 0.00821734 ... 0.00049222 0.00044217 0.00047735]
 [0.00491441 0.00179882 0.00475018 ... 0.00060894 0.00064152 0.00063278]
 [0.00147672 0.00109679 0.00081337 ... 0.00072816 0.00067932 0.00069495]]
Input -->  [6, 37, 347, 28]
1/1 [==============================] - 0s 26ms/step
Prediction --> [[0.01500825 0.00495367 0.00821734 ... 0.00049222 0.00044217 0.00047735]
 [0.00491441 0.00179882 0.00475018 ... 0.00060894 0.00064152 0.00063278]
 [0.00147671 0.00109679 0.00081337 ... 0.00072816 0.00067932 0.00069495]
 [0.00305042 0.00169052 0.00158334 ... 0.0006775  0.00073036 0.00066662]]
Input -->  [6, 37, 347, 28, 894]
1/1 [=============================

In [39]:
#El texto final generado

generated_text

'deep convolutional image recurrent rnn, supporting difficult, followed operations, data intermediate system, under et. normalized supervised that networks. foundation prone consistently unrealistic'

## Ejercicios

A. Realiza las siguientes modificaciones a la red y comenta los efectos que percibes en a) Valor de pérdida en las épocas. b) Tiempo de entrenamiento.
c) Calidad percibida del texto final generado.

1. Modifica la longitud de las secuencias de input.

2. Modifica la cantidad de unidades de la capa LSTM.

3. Modifica la cantidad de épocas de entrenamiento.

4. Modifica la temperatura de sampling.

5. Agrega una segunda capa recurrente LSTM (en ese caso la primer capa debe tener el parámetro return_sequences=True) ¿Percibes mejoras en relación a simplemente aumentar el tamaño de la capa?

B. Presenta 3 ejemplos de texto generado por tu red. Puedes utilizar diferentes palabras de inicialización.